**Table of contents**<a id='toc0_'></a>    
- [N2 Excited State Data](#toc1_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[N2 Excited State Data](#toc0_)
**first testing if jupyer is a good notetaking tool**


| state  | avdz             | avtz             |
| ------ | ---------------- | ---------------- |
| gs     | -109.4869509(85) | -109.534973(17)* |
| 1Pig   | -109.145116(82)  | -109.19543(16)** |
| 1Sigu- | -109.12030(20)   |                  |
| 1Delu  | -109.107187(52)  |                  |
| 3Sigu+ | -109.201567(40)  |                  |
| 3Pig   | -109.192041(41)  |                  |
| 3Delu  | -109.156050(34)  |                  |

just use pandoc style citations [@haupt2022; @laughlinTheory2000] and worry about matching citations when building in LaTeX (use pandoc to convert these)

!cat table.md


In [1]:
from IPython.display import Markdown, display
import pandas as pd
import uncertainties
from uncertainties import ufloat_fromstr
import string



display_csv = lambda f: display(Markdown(pd.read_csv(f,comment="#").to_markdown(index=False)))
display_md = lambda f: display(Markdown(open(f).read()))

In [2]:
import pandas as pd

from uncertainties import unumpy
from uncertainties import ufloat, ufloat_fromstr
from uncertainties.core import UFloat
from math import isnan

def format_ufloat(x):
    if isinstance(x, type(ufloat(1, 1))):
        return f"{x:+.1uS}"  # Uses the shorthand format
    return x

def col2ufloat(col):
    return [ufloat_fromstr("nan") if pd.isna(x) else ufloat_fromstr(x) for x in col]

def display_formatted(df):
    # Create a copy to avoid modifying original
    display_df = df.copy()

    # Format ufloat columns
    for col in df.columns:
        if isinstance(df[col].iloc[0], UFloat):
            display_df[col] = df[col].apply(lambda x: f"{x:+.1uS}")

    display(display_df)

def get_state_values(filename):
    # read in data
    df = pd.read_csv(filename, comment='#', skipinitialspace=True)
    df = df.map(lambda x: x.strip() if isinstance(x, str) else x) # strip data
    df.rename(columns=lambda x: x.strip(), inplace=True) # strip headers too

    # handle NaNs as ufloats as well
    df["avdz"] = col2ufloat(df["avdz"])
    df["avtz"] = col2ufloat(df["avtz"])


    display_formatted(df)

    # Get gs values as a series
    gs_values = df[df['state'] == 'gs'].iloc[0]
    # display(gs_values)

    # Create new dataframe excluding gs
    diff_df = df[df['state'] != 'gs'].copy()

    cols_to_diff = df.columns[1:]  # All columns except first
    for col in cols_to_diff:
        # display(diff_df[col])
        # display(gs_values[col])
        diff_df[col] = diff_df[col] - gs_values[col]
    for col in diff_df.columns[1:]:
        diff_df[col] *= 1000
    display_formatted(diff_df)

    return df, diff_df
    # type(df["avdz"][1])

In [3]:
df_casscf, dfdiff_casscf = get_state_values("N2_casscf_states.csv")

,state,avdz,avtz
0,gs,-109.478074(5),-109.53224(2)
1,1Pig,-109.13731(3),-109.19022(8)
2,1Sigu-,-109.11155(2),-109.16727(2)
3,1Delu,-109.10054(5),-109.1556(1)
4,3Sigu+,-109.19537(2),-109.24825(4)
5,3Pig,-109.18524(1),-109.23767(2)
6,3Delu,-109.15214(2),-109.20642(4)


,state,avdz,avtz
1,1Pig,+340.76(4),+342.02(8)
2,1Sigu-,+366.53(2),+364.97(3)
3,1Delu,+377.53(5),+376.7(1)
4,3Sigu+,+282.71(2),+283.99(5)
5,3Pig,+292.84(1),+294.57(3)
6,3Delu,+325.93(2),+325.81(5)


In [4]:
df_exp = pd.read_csv("N2_benchmark.csv", comment='#', skipinitialspace=True)
df_exp = df_exp.map(lambda x: x.strip() if isinstance(x, str) else x) # strip data
df_exp.rename(columns=lambda x: x.strip(), inplace=True) # strip headers too
df_exp["exp1"] *= 37
df_exp["exp2"] *= 37
df_exp["theory"] *= 37
# df.to_csv("N2_benchmark_converted.csv", index=False)

In [7]:
display_csv("N2_loos.csv")
display_formatted(dfdiff_casscf)
display_formatted(df_exp)
# display_formatted(df_casscf)

|  state     |  avdz        |  avtz        |  avqz    |
|:-----------|:-------------|:-------------|:---------|
| 1Pig       | 345.8(0)     | 343.2(0)     | 343.2(0) |
| 1Sigu-     | 369.3(0)     | 363.1(0)     | 364.6(0) |
| 1Delu      | 383.3(0)     | 378.2(0)     | 378.9(0) |
| 3Sigu+     | 283.0(0)     | 283.0(0)     | 284.4(0) |
| 3Pig       | 295.8(0)     | 294.4(0)     | 295.1(0) |
| 3Delu      | 329.3(0)     | 326.0(0)     | 326.3(0) |

,state,avdz,avtz
1,1Pig,+340.76(4),+342.02(8)
2,1Sigu-,+366.53(2),+364.97(3)
3,1Delu,+377.53(5),+376.7(1)
4,3Sigu+,+282.71(2),+283.99(5)
5,3Pig,+292.84(1),+294.57(3)
6,3Delu,+325.93(2),+325.81(5)


,state,exp1,exp2,theory
0,1Pig,344.47,344.47,342.99
1,1Sigu-,367.04,367.04,373.33
2,1Delu,379.99,379.99,389.98
3,3Sigu+,286.75,286.75,279.72
4,3Pig,297.48,297.48,297.85
5,3Delu,328.56,328.56,330.41
